<a href="https://colab.research.google.com/github/bobman38/jira-notebooks/blob/main/jira_roadmap_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roadmap Plan

This document will help you to plan a release for your project on Jira. It show you all `Epic` for a specific future version, and give you the consumed time and estimated time (rely on a custom field).

It's a [Jupyter Notebook](https://jupyter.org/) document. Each code cell must been run in the order of the document.

# Prerequisiste

First let's import the `jira` python package and import the needed libraries.  

> Even if the result of this cell is not shown, it is needed to run this cell too or the other cells will fail.

In [ ]:
#@title Install Jira package and imports
%%capture
!pip install jira;
from jira import JIRA
from IPython.display import HTML
import getpass
import json
import pandas as pd

# Connect to  Jira

The next cell will connect to Jira and create the `jira` that will be used later. The `uid` field must be your Jira email, and the `pswd` field must be an API token that you can create [here](https://id.atlassian.com/manage-profile/security/api-tokens).  The `pswd` is asked only when the the cell is launched by security (never stored in the document).

In [ ]:
# Init Jira connection
host = "https://yoursite.atlassian.net" #@param {type:"string"}
uid = "user@domain.ltd" #@param {type:"string"}
pswd = getpass.getpass('Password:')
jira = JIRA(server=host, basic_auth=(uid, pswd))
print("Connected to Jira !")

# Epic Table

The next cell will generate a table for all epic on a specific project and a specific version. The `estimatedTime` is sumed in the footer of the table. _Take note that here we are relying on a custom field (`customfield_10624`), you will have to adapt that on your Jira instance._

In [ ]:
# Show epics !
project = "MYPROJECT" #@param {type:"string"}
version = "1.0.0" #@param {type:"string"}
issues = jira.search_issues('project = "' + project + '" and issuetype = "Epic" and (fixversion = ' + version+ ') ORDER BY key DESC', maxResults=0,startAt=0)
formated_issues = list()
for issue in issues:
    # Compute estimated for each epic
    estimated = 0
    if(issue.fields.customfield_10624):
        estimated = float(issue.fields.customfield_10624.replace("d", ""))
    # add epic to list
    formated_issues.append([issue.key, issue.fields.components, issue.fields.status.name, issue.fields.priority.name, issue.fields.summary, estimated])
df = pd.DataFrame(formated_issues, columns=["id", "components", "status", "priority", "summary", "estimatedTime"])
df = df.sort_values(["status", "estimatedTime"])
df['id'] = df['id'].apply(lambda x: '<a href="https://myurl.atlassian.net/browse/{0}">{0}</a>'.format(x))
df = df.append(pd.DataFrame([[df.id.count(), df.estimatedTime.sum()]], index = ["Total"], columns=["id", "estimatedTime"]))
HTML(df.to_html(escape=False))